Several issues:
1. test.csv file does not have ground-truth (price range in test.csv file). So I split train.csv file into train and test data in 0.8:0.2 ratio. 
2. Should we normalize the data for columns (data type of numerical type) (not binary (0, 1))? z-score normalization. min_max normalization? 
3. I did agglomerate cluster, kmeans++ (used from my homework 2), decisionTree learned from online course.
4. Could someone do OPTICS/DBSCAN? i think it is possible?
5. f1 score: I am not sure if the average type is micro, macro or weighted. I chose micro. 
5. Any other idea? Write-up and other method we can add-on?

This is preliminary start. I haven't looked into patterns. 



In [ ]:
import pandas as pd
import numpy as np

print("pandas version: ", pd.__version__)
print("numpy version: ", np.__version__)

pandas version:  1.3.5
numpy version:  1.21.2


In [2]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
print(test_data.shape)
print(train_data.shape)

(1000, 21)
(2000, 21)


Print out description of train data and test data to check validity of data. Check data to ensure that there is missing value. 

In [35]:
print(test_data.describe())
# issue of test_data is that it does not have ground truth column. no price range column. 

                id  battery_power         blue  clock_speed     dual_sim  \
count  1000.000000    1000.000000  1000.000000  1000.000000  1000.000000   
mean    500.500000    1248.510000     0.516000     1.540900     0.517000   
std     288.819436     432.458227     0.499994     0.829268     0.499961   
min       1.000000     500.000000     0.000000     0.500000     0.000000   
25%     250.750000     895.000000     0.000000     0.700000     0.000000   
50%     500.500000    1246.500000     1.000000     1.500000     1.000000   
75%     750.250000    1629.250000     1.000000     2.300000     1.000000   
max    1000.000000    1999.000000     1.000000     3.000000     1.000000   

                fc       four_g   int_memory        m_dep   mobile_wt  ...  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.00000  ...   
mean      4.593000     0.487000    33.652000     0.517500   139.51100  ...   
std       4.463325     0.500081    18.128694     0.280861    34.85155  ...   
min

In [34]:
print(train_data.describe())

       battery_power       blue  clock_speed     dual_sim           fc  \
count    2000.000000  2000.0000  2000.000000  2000.000000  2000.000000   
mean     1238.518500     0.4950     1.522250     0.509500     4.309500   
std       439.418206     0.5001     0.816004     0.500035     4.341444   
min       501.000000     0.0000     0.500000     0.000000     0.000000   
25%       851.750000     0.0000     0.700000     0.000000     1.000000   
50%      1226.000000     0.0000     1.500000     1.000000     3.000000   
75%      1615.250000     1.0000     2.200000     1.000000     7.000000   
max      1998.000000     1.0000     3.000000     1.000000    19.000000   

            four_g   int_memory        m_dep    mobile_wt      n_cores  ...  \
count  2000.000000  2000.000000  2000.000000  2000.000000  2000.000000  ...   
mean      0.521500    32.046500     0.501750   140.249000     4.520500  ...   
std       0.499662    18.145715     0.288416    35.399655     2.287837  ...   
min       0.00000

Check if there is duplicate row in test data and train data. 

In [18]:
res = True
for i in test_data.duplicated():
    if i:
        print("row i:", i, "is duplicate row")
        res = False

if res:
    print("There is no duplicate value in test data. ")


There is no duplicate value in test data. 


In [19]:
res = True
for i in train_data.duplicated():
    if i:
        print("row i:", i, "is duplicate row")
        res = False

if res:
    print("There is no duplicate value in train data. ")


There is no duplicate value in train data. 


Machine learning model from codewithmosh.com. password protected. 
reference: https://codewithmosh.com/courses/enrolled/417695


In [50]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import sklearn
print("sklearn version: ", sklearn.__version__)

model = DecisionTreeClassifier()
X = train_data.drop(columns=['price_range'])
Y = train_data['price_range']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
model.fit(X_train, Y_train)



sklearn version:  1.0.2


DecisionTreeClassifier()

In [46]:
# Machine learning model from codewithmosh.com. password protected. 
# reference: https://codewithmosh.com/courses/enrolled/417695

import joblib
joblib.dump(model,"decision_tree_mobile_phone.joblib")



['decision_tree_mobile_phone.joblib']

In [47]:
model = joblib.load("decision_tree_mobile_phone.joblib")
predictions = model.predict(X_test)

In [48]:
# f-measure reference
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

from sklearn.metrics import f1_score
f1_score_decision_tree = f1_score(Y_test,predictions,average = 'micro')  # options include micro, marco, weighted. 
print('f1 score of decision tree: ', f1_score_decision_tree)

f1 score of decision tree:  0.8425


price range: <br>
0: low cost  <br>
1: medium cost  <br>
2: high cost  <br>
3: very high cost  <br>
cluster number is 4


In [52]:
# agglomerate cluster (n = 4)
from sklearn.cluster import AgglomerativeClustering as ac

In [55]:
# single link
prediction_single_link = ac(n_clusters=4,linkage='single').fit_predict(X)
f1_score_single_link = f1_score(Y, prediction_single_link, average = 'micro')
print('f1 score of single link: ', f1_score_single_link)

f1 score of single link:  0.249


why the F1 score is only 0.249??
Should we normalize the data? on values not binary (0 or 1). 

In [57]:
# complete link
prediction_complete_link = ac(n_clusters=4,linkage='complete').fit_predict(X)
f1_score_complete_link = f1_score(Y, prediction_complete_link, average = 'micro')
print('f1 score of complete link: ', f1_score_complete_link)

f1 score of complete link:  0.2445


In [ ]:
# k-means++ (cluster n = 4)

In [59]:
from sklearn.cluster import KMeans

Kmeans study on unnormalized data:

In [61]:
predictions_kmeans = KMeans(n_clusters=4).fit_predict(X)
f1_score_kmeans = f1_score(Y, predictions_kmeans, average = 'micro')
print('f1 score of kmeans: ', f1_score_kmeans)

f1 score of kmeans:  0.3695


In [ ]:
# DBSCAN/OPTICS
# I did DBSCAN/OPTICS manually in the previous homework so I do not have code with me. 
